
# PhoBERT 5-Classes Sentiment Inference (Notebook)

Notebook hoá từ script inference của bạn.  
- Hỗ trợ cả **.txt** (mỗi dòng 1 câu) và **.csv** (cột `text`, tuỳ chọn `label`)  
- Tiền xử lý giống lúc train (normalize + optional word segmentation)  
- Tuỳ chọn **neutral penalty** (điều chỉnh logit lớp *neutral*)  
- Xuất **CSV** dự đoán (+ xác suất) nếu muốn

> **Gợi ý chạy:** Sửa `CFG` ở ô **Config** (đường dẫn model/input/output), sau đó chạy lần lượt các ô từ trên xuống.


## Config

In [28]:

from types import SimpleNamespace

# === Sửa lại cấu hình tại đây ===
CFG = SimpleNamespace(
    model_dir="/home/dat/llm_ws/phobert/phobert_5cls_clean",       # Thư mục model đã save_model()
    input_txt="",                                           # File .txt (mỗi dòng 1 câu). Để "" nếu dùng CSV
    input_csv="/home/dat/llm_ws/data/test/vn_product_reviews_test_100_challenge.csv",  # File .csv (cột 'text', optional 'label')
    max_len=160,
    batch_size=64,
    show=20,                                                # Số dòng print ra
    normalize=True,
    use_seg=False,                                          # Bật word segmentation nếu lúc train có bật
    neutral_penalty=0.0,                                    # Ví dụ: -0.2 để "phạt" neutral
    out_csv=""                                              # Đường dẫn CSV để lưu (ví dụ: "/home/dat/llm_ws/out/preds.csv"); "" nếu không lưu
)
print(CFG)


namespace(model_dir='/home/dat/llm_ws/phobert/phobert_5cls_clean', input_txt='', input_csv='/home/dat/llm_ws/data/test/vn_product_reviews_test_100_challenge.csv', max_len=160, batch_size=64, show=20, normalize=True, use_seg=False, neutral_penalty=0.0, out_csv='')


## Imports

In [29]:

import os, sys, re
import numpy as np
import pandas as pd
import torch
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification

LABELS_5 = ["very_negative","negative","neutral","positive","very_positive"]
LBL2ID = {l:i for i,l in enumerate(LABELS_5)}
ID2LBL = {i:l for l,i in LBL2ID.items()}


## Tiền xử lý & Helpers

In [30]:

# ==== same normalize as training ====
EMO_POS = ["🤩","🥰","😍","❤️","👍","😎","👌","✨","🔥","💯"]
EMO_NEG = ["😱","😡","🤬","💩","👎","😤","😞","😭"]

def normalize_text(s: str) -> str:
    s = str(s).strip()
    for e in EMO_POS: s = s.replace(e, " EMO_POS ")
    for e in EMO_NEG: s = s.replace(e, " EMO_NEG ")
    repl = {
        "vl": "rất", "okeee": "ok", "ưng": "rất thích",
        "siêu siêu": "rất", "siêu thất vọng": "rất thất vọng",
        "mãi đỉnh": "rất tốt", "best of best": "rất tốt", "best choice": "rất tốt",
        "đỉnh của chóp": "rất tốt",
    }
    for k,v in repl.items():
        s = re.sub(rf"\b{re.escape(k)}\b", v, s, flags=re.IGNORECASE)
    return s

def maybe_segment(text, use_seg=False):
    if not use_seg: return text
    try:
        from underthesea import word_tokenize
        return word_tokenize(text, format="text")
    except Exception as e:
        print("[Cảnh báo] Không thể import underthesea. Tắt use_seg hoặc cài đặt thư viện. Lỗi:", e)
        return text

def softmax(x):
    x = x - np.max(x, axis=-1, keepdims=True)
    e = np.exp(x)
    return e / np.sum(e, axis=-1, keepdims=True)

def load_texts_from_txt(path):
    with open(path, "r", encoding="utf-8") as f:
        lines = [l.strip() for l in f if l.strip()]
    return lines, None  # no labels

def load_texts_from_csv(path):
    df = pd.read_csv(path)
    assert "text" in df.columns, "CSV phải có cột 'text'"
    texts = df["text"].astype(str).tolist()
    labels = None
    if "label" in df.columns:
        labels = [LBL2ID[l] if l in LBL2ID else None for l in df["label"].astype(str)]
    return texts, labels, df

def batched(iterable, n):
    for i in range(0, len(iterable), n):
        yield iterable[i:i+n]

def apply_preproc(texts, normalize=True, use_seg=False):
    out = []
    for t in texts:
        s = normalize_text(t) if normalize else t
        s = maybe_segment(s, use_seg=use_seg)
        out.append(s)
    return out

def maybe_apply_class_bias(logits, bias_vec):
    # bias_vec: list of floats length=5, added to logits (logit adjustment)
    if bias_vec is None: return logits
    b = np.array(bias_vec, dtype=np.float32).reshape(1, -1)
    return logits + b


## Load model/tokenizer

In [31]:

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)
tok = AutoTokenizer.from_pretrained(CFG.model_dir, use_fast=False)
mdl = AutoModelForSequenceClassification.from_pretrained(CFG.model_dir).to(device).eval()
print("Loaded model from:", CFG.model_dir)


Device: cuda
Loaded model from: /home/dat/llm_ws/phobert/phobert_5cls_clean


## Nạp dữ liệu đầu vào (.txt hoặc .csv)

In [32]:

texts, labels, df_src = None, None, None
if CFG.input_txt:
    texts, _ = load_texts_from_txt(CFG.input_txt)
elif CFG.input_csv:
    texts, labels, df_src = load_texts_from_csv(CFG.input_csv)
else:
    raise SystemExit("Cần cấu hình input_txt hoặc input_csv")

if len(texts) == 0:
    raise SystemExit("Không có câu nào để dự đoán.")

print(f"Số dòng input: {len(texts)}")


Số dòng input: 100


## Tiền xử lý (giống train)

In [33]:

texts_proc = apply_preproc(texts, normalize=CFG.normalize, use_seg=CFG.use_seg)
print("Ví dụ sau tiền xử lý:", texts_proc[0] if texts_proc else "(empty)")


Ví dụ sau tiền xử lý: loa hỏng ngay lần 1, giao hàng kẹt mãi.


## Dự đoán theo batch

In [34]:

all_logits = []
with torch.no_grad():
    for chunk in batched(texts_proc, CFG.batch_size):
        enc = tok(chunk, truncation=True, padding=True, max_length=CFG.max_len, return_tensors="pt")
        enc = {k: v.to(device) for k, v in enc.items()}
        logits = mdl(**enc).logits.detach().cpu().numpy()
        all_logits.append(logits)
logits = np.concatenate(all_logits, axis=0)

# optional class-bias (e.g., penalize neutral)
bias = None
if CFG.neutral_penalty != 0.0:
    bias = [0.0, 0.0, CFG.neutral_penalty, 0.0, 0.0]  # add to logits
logits = maybe_apply_class_bias(logits, bias)

probs = softmax(logits)
pred_ids = probs.argmax(-1)
pred_labels = [ID2LBL[int(i)] for i in pred_ids]
pmax = probs.max(axis=1)

print("Hoàn tất dự đoán.")


Hoàn tất dự đoán.


## Hiển thị nhanh (first N)

In [35]:

show_n = min(len(texts), CFG.show)
print("\n=== Predictions (first N) ===")
for t, pi, p in zip(texts[:show_n], pred_ids[:show_n], pmax[:show_n]):
    print(f"{ID2LBL[int(pi)]:14s}  {p:0.3f}  | {t}")
if len(texts) > show_n:
    print(f"... ({len(texts)-show_n} more)")



=== Predictions (first N) ===
very_negative   0.951  | loa hỏng ngay lần 1, giao hàng kẹt mãi.
neutral         0.853  | robot hut bui không nổi bật, giá tạm ổn.
positive        0.958  | Sản phẩm robot hut bui xịn sò vl, dùng khá ok.
positive        0.958  | Sản phẩm laptop đỉnh cao 🤩, dùng khá ok.
very_positive   0.955  | tai nghe không chê vào đâu đc, quá yêu.
negative        0.956  | Mình thấy chuot khó chịu vc, chất lượng chưa ổn.
neutral         0.877  | chuot ổn để dùng văn phòng, giá tạm ổn.
neutral         0.881  | loa tạm dc, giá tạm ổn.
very_negative   0.952  | laptop thảm họa 😱, giao hàng kẹt mãi.
positive        0.957  | Sản phẩm man hinh mượt mà 10/10, dùng khá ok.
negative        0.957  | Mình thấy man hinh khó chịu vc, chất lượng chưa ổn.
neutral         0.871  | chuot không nổi bật, giá tạm ổn.
negative        0.956  | Mình thấy chuot rep chậm, chất lượng chưa ổn.
positive        0.958  | Sản phẩm robot hut bui đỉnh cao 🤩, dùng khá ok.
positive        0.959  | Sản phẩm 

## Metrics (nếu có nhãn hợp lệ trong CSV)

In [36]:

if labels is not None and any(l is not None for l in labels):
    idx = [i for i,l in enumerate(labels) if l is not None]
    y_true = np.array([labels[i] for i in idx], dtype=int)
    y_pred = pred_ids[idx]
    print("\n=== Metrics (on rows with valid labels) ===")
    print(f"Accuracy : {accuracy_score(y_true, y_pred):.4f}")
    print(f"Macro F1 : {f1_score(y_true, y_pred, average='macro'):.4f}")
    print("\nConfusion matrix (rows=true, cols=pred):")
    print(confusion_matrix(y_true, y_pred))
    print("\nClassification report:")
    print(classification_report(y_true, y_pred, target_names=LABELS_5, digits=4))
else:
    print("Không có cột label hoặc không có nhãn hợp lệ => bỏ qua tính metrics.")



=== Metrics (on rows with valid labels) ===
Accuracy : 1.0000
Macro F1 : 1.0000

Confusion matrix (rows=true, cols=pred):
[[20  0  0  0  0]
 [ 0 20  0  0  0]
 [ 0  0 20  0  0]
 [ 0  0  0 20  0]
 [ 0  0  0  0 20]]

Classification report:
               precision    recall  f1-score   support

very_negative     1.0000    1.0000    1.0000        20
     negative     1.0000    1.0000    1.0000        20
      neutral     1.0000    1.0000    1.0000        20
     positive     1.0000    1.0000    1.0000        20
very_positive     1.0000    1.0000    1.0000        20

     accuracy                         1.0000       100
    macro avg     1.0000    1.0000    1.0000       100
 weighted avg     1.0000    1.0000    1.0000       100



## Lưu CSV (tuỳ chọn)

In [37]:

if CFG.out_csv:
    if df_src is None:
        df_out = pd.DataFrame({"text": texts})
    else:
        df_out = df_src.copy()
    df_out["_pred"] = pred_labels
    df_out["_pmax"] = pmax
    for i, name in enumerate(LABELS_5):
        df_out[f"prob_{name}"] = probs[:, i]
    os.makedirs(os.path.dirname(CFG.out_csv) or ".", exist_ok=True)
    df_out.to_csv(CFG.out_csv, index=False, encoding="utf-8")
    print(f"[Saved] {CFG.out_csv}")
else:
    print("CFG.out_csv rỗng => không lưu CSV.")


CFG.out_csv rỗng => không lưu CSV.


## Demo nhanh (một câu on-the-fly)

In [42]:

demo_text = "san pham qua toi te"   
demo_proc = apply_preproc([demo_text], normalize=CFG.normalize, use_seg=CFG.use_seg)

with torch.no_grad():
    enc = tok(demo_proc, truncation=True, padding=True, max_length=CFG.max_len, return_tensors="pt")
    enc = {k: v.to(device) for k, v in enc.items()}
    logit = mdl(**enc).logits.detach().cpu().numpy()[0]

if CFG.neutral_penalty != 0.0:
    logit = maybe_apply_class_bias(logit[None, :], [0.0, 0.0, CFG.neutral_penalty, 0.0, 0.0])[0]

prob = softmax(logit[None, :])[0]
pred = ID2LBL[int(prob.argmax())]
print("Text:", demo_text)
print("Pred:", pred)
print("Probs:", {LABELS_5[i]: float(prob[i]) for i in range(len(LABELS_5))})


Text: san pham qua toi te
Pred: positive
Probs: {'very_negative': 0.013087497092783451, 'negative': 0.011440116912126541, 'neutral': 0.00748001504689455, 'positive': 0.9563746452331543, 'very_positive': 0.011617729440331459}
